# Cuaderno 14: Agrupar y agregar información

Una tarea esencial dentro del análisis de datos es la *agregación*, que consiste en la sintetización conjuntos de datos muy grandes a un cierto nivel, para extraer información útil.

En los Cuadernos 10 y 11 hemos revisado algunas funciones de agregación disponibles para `Series` y `DataFrames`: `sum()`, `min()`, `max()`, entre otras. En este cuaderno examinaremos cómo se pueden agrupar los datos en un DataFrame de acuerdo a uno o más criterios y aplicar funciones de agregación a nivel de los grupos de datos.

Empezamos por importar los módulos de `pandas`y `numpy`:

In [1]:
# importar pandas y NumPy
import numpy as np
import pandas as pd

## Preparación de los conjuntos de datos

Empezaremos por preparar los conjuntos de datos que vamos a requerir en este cuaderno. Vamos a trabajar con datos del registro de defunciones generales del Ecuador en el año 2019, obtenido del sitio web del Instituto Nacional de Estadística y Censos del Ecuador (INEC) <https://www.ecuadorencifras.gob.ec/defunciones-generales-2019/>. De este archivo en formato `csv` nos interesan solamente algunas variables:
* `sexo`: sexo de la persona fallecida (1: Masculino, 2: Femenino)
* `prov_fall`: código de la provincia en la que se registró el fallecimiento
* `cant_fall`: código del cantón en el que se registró en fallecimiento
* `mor_vio`: causa del fallecimiento, en el caso de las muertes violentas (número del 1 al 6, en blanco significa muerte natural).

Importamos estas columnas, sustituyendo los espacios en blanco por `NaN`:

In [2]:
# importar datos del registro de defunciones de 2019 del INEC
dfmuertes2019 = pd.read_csv('BDD_EDG_2019.csv', sep=';', 
                            usecols=['sexo', 'prov_fall', 'cant_fall', 'mor_viol'], 
                            na_values=' ') # espacios en blanco se tratarán como NaN
display(dfmuertes2019)

,sexo,prov_fall,cant_fall,mor_viol
0,1,9,901,1.0
1,1,17,1701,NaN
2,1,17,1701,2.0
3,1,17,1701,NaN
4,1,9,901,NaN
...,...,...,...,...
75350,2,17,1701,NaN
75351,2,9,906,NaN
75352,2,5,501,2.0
75353,2,9,901,NaN


Vamos a renombrar las columnas de `dfmuertes2019` con nombres más adecuados para nuestro ejemplo. Reemplazaremos además la columna `mor_viol` por una columna `violenta` en la que se indique (con 0 o 1) si se trata de una muerta violenta o no:

In [3]:
# sustituir nombres de las columnas
dfmuertes2019.rename(columns= {'prov_fall' : 'cod_provincia', 'cant_fall' : 'cod_canton', 'mor_viol' : 'violenta'}, 
                     inplace= True)
# sustituir valores de NaN por cero
dfmuertes2019.fillna(0, inplace=True)
# sustituir valores mayores a cero en la columna violenta por 1's
dfmuertes2019.loc[dfmuertes2019['violenta']>0,'violenta'] = 1
dfmuertes2019= dfmuertes2019.astype({'violenta':'int'})
display(dfmuertes2019)

,sexo,cod_provincia,cod_canton,violenta
0,1,9,901,1
1,1,17,1701,0
2,1,17,1701,1
3,1,17,1701,0
4,1,9,901,0
...,...,...,...,...
75350,2,17,1701,0
75351,2,9,906,0
75352,2,5,501,1
75353,2,9,901,0


Importaremos ahora en el DataFrame `dfcantones` la información de los nombres de cada cantón y de su población estimada en el año 2019. Estos datos son extraídos del archivo `proyeccion_cantonal_total_2010-2020.xlsx`, descargado del sitio web del Instituto Nacional de Estadística y Censos del Ecuador (INEC) <https://www.ecuadorencifras.gob.ec/proyecciones-poblacionales/>. 

In [4]:
# importar datos de proyección poblacional por cantones del INEC
dfcantones = pd.read_excel('proyeccion_cantonal_total_2010-2020.xlsx', skiprows=2,
                            usecols='A:B,L') 
display(dfcantones)

,Código,Nombre de canton,2019
0,101,CUENCA,625775
1,102,GIRON,13074
2,103,GUALACEO,48702
3,104,NABON,17250
4,105,PAUTE,28985
...,...,...,...
219,2402,LIBERTAD,115952
220,2403,SALINAS,92017
221,9001,LAS GOLONDRINAS,7370
222,9003,MANGA DEL CURA,26061


Nuevamente, cambiamos los nombres de las columnas de `dfcantones` por valores más adecuados:

In [5]:
dfcantones.rename(columns={'Código' : 'cod_canton', 'Nombre de canton' : 'canton', 
                           2019 : 'poblacion'}, inplace= True)
display(dfcantones)

,cod_canton,canton,poblacion
0,101,CUENCA,625775
1,102,GIRON,13074
2,103,GUALACEO,48702
3,104,NABON,17250
4,105,PAUTE,28985
...,...,...,...
219,2402,LIBERTAD,115952
220,2403,SALINAS,92017
221,9001,LAS GOLONDRINAS,7370
222,9003,MANGA DEL CURA,26061


Empleando la función `merge`, cruzamos la información de los DataFrames `dfmuertes2019` y `dfcantones` para obtener el DataFrame `dfmuertes2019ext`, que contiene el registro de defunciones con información del nombre del cantón y su población. 

Notar que en este proceso se pierden algunos registros, debido a que sus códigos de cantón no aparecen en `dfcantones`. Este tipo de situaciones ocurren con frecuencia al combinar información de distintas fuentes, y su causa debe determinarse (y de ser posible, corregirse). Para este ejemplo, sin embargo, vamos a trabajar, por simplicidad, con los registros restantes.

In [6]:
dfmuertes2019ext= pd.merge(dfmuertes2019, dfcantones, on='cod_canton')
display(dfmuertes2019ext)

,sexo,cod_provincia,cod_canton,violenta,canton,poblacion
0,1,9,901,1,GUAYAQUIL,2698077
1,1,9,901,0,GUAYAQUIL,2698077
2,1,9,901,0,GUAYAQUIL,2698077
3,1,9,901,1,GUAYAQUIL,2698077
4,1,9,901,0,GUAYAQUIL,2698077
...,...,...,...,...,...,...
75215,2,16,1604,0,ARAJUNO,7989
75216,1,16,1603,0,SANTA CLARA,4110
75217,1,16,1603,1,SANTA CLARA,4110
75218,2,16,1603,0,SANTA CLARA,4110


Finalmente, creamos una serie `provincias` con los nombres de las 24 provincias del país, indexados por sus respectivos códigos:

In [7]:
provincias = pd.Series(['Azuay', 'Bolívar', 'Cañar', 'Carchi', 'Cotopaxi', 'Chimborazo',
                        'El Oro','Esmeraldas', 'Guayas', 'Imbabura', 'Loja', 'Los Ríos',
                        'Manabí', 'Morona Santiago', 'Napo', 'Pastaza', 'Pichincha',
                        'Tungurahua', 'Zamora Chinchipe', 'Galápagos', 'Sucumbíos',
                        'Orellana', 'Santo Domingo de los Tsáchilas', 'Santa Elena'], index=range(1,25), 
                        name='nom_provincia')
print(provincias)

1                              Azuay
2                            Bolívar
3                              Cañar
4                             Carchi
5                           Cotopaxi
6                         Chimborazo
7                             El Oro
8                         Esmeraldas
9                             Guayas
10                          Imbabura
11                              Loja
12                          Los Ríos
13                            Manabí
14                   Morona Santiago
15                              Napo
16                           Pastaza
17                         Pichincha
18                        Tungurahua
19                  Zamora Chinchipe
20                         Galápagos
21                         Sucumbíos
22                          Orellana
23    Santo Domingo de los Tsáchilas
24                       Santa Elena
Name: nom_provincia, dtype: object


Al cruzar el DataFrame `dfmuertes2019ext` con la serie `provincias` usando el método `join`, incorporamos la información de los nombres de provincias en el registro de las defunciones: 

In [8]:
dfmuertes2019ext = dfmuertes2019ext.join(provincias, on='cod_provincia')
display(dfmuertes2019ext)

,sexo,cod_provincia,cod_canton,violenta,canton,poblacion,nom_provincia
0,1,9,901,1,GUAYAQUIL,2698077,Guayas
1,1,9,901,0,GUAYAQUIL,2698077,Guayas
2,1,9,901,0,GUAYAQUIL,2698077,Guayas
3,1,9,901,1,GUAYAQUIL,2698077,Guayas
4,1,9,901,0,GUAYAQUIL,2698077,Guayas
...,...,...,...,...,...,...,...
75215,2,16,1604,0,ARAJUNO,7989,Pastaza
75216,1,16,1603,0,SANTA CLARA,4110,Pastaza
75217,1,16,1603,1,SANTA CLARA,4110,Pastaza
75218,2,16,1603,0,SANTA CLARA,4110,Pastaza


Notar que sobre este DataFrame pueden aplicarse las funciones de agregación conocidas:

In [9]:
# número total de defunciones
print('Cuenta de valores por filas:')
print(dfmuertes2019ext.count())
print('---')

# número total de muertes violentas (suma de la columna violenta)
print('Total muertes violentas : {}'.format(dfmuertes2019ext['violenta'].sum()))
print('---')

# número total de muertes del sexo masculino
print('Total muertes hombres : {}'.format(dfmuertes2019ext[dfmuertes2019ext['sexo']==1]['sexo'].count()))
print('---')

# número total de muertes en Guayaquil
print('Total muertes en cantón Guayaquil : {}'.format(
    dfmuertes2019ext[dfmuertes2019ext['canton']=='GUAYAQUIL']['canton'].count()))


Cuenta de valores por filas:
sexo             75220
cod_provincia    75220
cod_canton       75220
violenta         75220
canton           75220
poblacion        75220
nom_provincia    75220
dtype: int64
---
Total muertes violentas : 9116
---
Total muertes hombres : 41636
---
Total muertes en cantón Guayaquil : 16499


## Seccionar, aplicar, combinar

Para el agrupamiento y agregación, en Pandas se emplea una estrategia conocida como *seccionar, aplicar y combinar* (*split, apply, combine*). Primero, el DataFrame es seccionado en varios DataFrames de acuerdo a algún criterio de agrupación. Luego, se aplica una función de agregación sobre cada DataFrame. Finalmente, los resultados de las agregaciones son combinados en un nuevo DataFrame. La siguiente figura ilustra este procedimiento:

![title](split_apply_combine.png)

El método `groupby` se encarga de *seccionar* el DataFrame de acuerdo a los valores de una columna. Comúnmente, este método es combinado con alguna función de agregación, la cual se *aplica* a cada sección del DataFrame. Los resultados de esta función se *combinan* luego automáticamente en un único DataFrame de respuesta.

Por ejemplo, suponer que se quiere contar el número de registros de fallecimientos por provincia:

In [10]:
# número de fallecidos por provincia
display(dfmuertes2019ext.groupby('nom_provincia').count())

# número de fallecidos por sexo
display(dfmuertes2019ext.groupby('sexo').count())

# muertes violentas vs. no violentas
display(dfmuertes2019ext.groupby('violenta').count())

,sexo,cod_provincia,cod_canton,violenta,canton,poblacion
nom_provincia,,,,,,
Azuay,4100,4100,4100,4100,4100,4100
Bolívar,853,853,853,853,853,853
Carchi,760,760,760,760,760,760
Cañar,1169,1169,1169,1169,1169,1169
Chimborazo,2451,2451,2451,2451,2451,2451
Cotopaxi,1934,1934,1934,1934,1934,1934
El Oro,3131,3131,3131,3131,3131,3131
Esmeraldas,1804,1804,1804,1804,1804,1804
Galápagos,47,47,47,47,47,47


,cod_provincia,cod_canton,violenta,canton,poblacion,nom_provincia
sexo,,,,,,
1,41636,41636,41636,41636,41636,41636
2,33584,33584,33584,33584,33584,33584


,sexo,cod_provincia,cod_canton,canton,poblacion,nom_provincia
violenta,,,,,,
0,66104,66104,66104,66104,66104,66104
1,9116,9116,9116,9116,9116,9116


En los ejemplos anteriores, la función de agregación `count` fue aplicada a *todas* las columnas de cada sección del DataFrame, retornando como resultado una fila de valores. Luego estas filas fueron combinadas en un DataFrame indexado por la columna que se utilizó para seccionar a `dfmuertes2019ext`.

En lugar de la función `count` puede usarse cualquier otra función de agregación:

In [11]:
# estos datos no tienen significado real: 
# se retorna el máximo valor de cada columna en cada provincia
display(dfmuertes2019ext.groupby('nom_provincia').max())

,sexo,cod_provincia,cod_canton,violenta,canton,poblacion
nom_provincia,,,,,,
Azuay,2,1,115,1,SIGSIG,625775
Bolívar,2,2,207,1,SAN MIGUEL,107590
Carchi,2,4,406,1,TULCAN,101234
Cañar,2,3,307,1,SUSCAL,85030
Chimborazo,2,6,610,1,RIOBAMBA,261360
Cotopaxi,2,5,507,1,SIGCHOS,202878
El Oro,2,7,714,1,ZARUMA,286120
Esmeraldas,2,8,807,1,SAN LORENZO,216901
Galápagos,2,20,2003,1,SANTA CRUZ,19852


La función `groupby` retorna un objeto del tipo `DataFrameGroupBy`. Este objeto representa al DataFrame seccionado de acuerdo a los valores de una columna. Puede pensarse en este objeto como en una colección de DataFrames, que tienen todos las mismas columnas.

In [12]:
print(type(dfmuertes2019ext.groupby('nom_provincia')))

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


Además de las funciones de agregación, un objeto `DataFrameGroupBy` presenta cierta funcionalidad básica que es útil en el procesamiento de datos. Al igual que en un `DataFrame`, puede usarse el operador `[]` para seleccionar una o varias columnas:

In [13]:
# número de fallecidos por provincia, elegimos arbitrariamente una sola columna
display(dfmuertes2019ext.groupby('nom_provincia')['cod_provincia'].count())

nom_provincia
Azuay                              4100
Bolívar                             853
Carchi                              760
Cañar                              1169
Chimborazo                         2451
Cotopaxi                           1934
El Oro                             3131
Esmeraldas                         1804
Galápagos                            47
Guayas                            21654
Imbabura                           2046
Loja                               2481
Los Ríos                           3712
Manabí                             6797
Morona Santiago                     534
Napo                                413
Orellana                            445
Pastaza                             356
Pichincha                         13355
Santa Elena                        1334
Santo Domingo de los Tsáchilas     2061
Sucumbíos                           673
Tungurahua                         2834
Zamora Chinchipe                    276
Name: cod_provincia, dtype

Es posible usar el objeto `DataFrameGroupBy` directamente en un lazo for, para iterar sobre los DataFrames de las distintas secciones, aunque esta es una operación poco usual:

In [14]:
# iterar por los DataFrames asociados los distintos sexos:
for (sexo, df) in dfmuertes2019ext.groupby('sexo'):
    print('Sexo: {}'.format(sexo))
    display(df)

Sexo: 1


,sexo,cod_provincia,cod_canton,violenta,canton,poblacion,nom_provincia
0,1,9,901,1,GUAYAQUIL,2698077,Guayas
1,1,9,901,0,GUAYAQUIL,2698077,Guayas
2,1,9,901,0,GUAYAQUIL,2698077,Guayas
3,1,9,901,1,GUAYAQUIL,2698077,Guayas
4,1,9,901,0,GUAYAQUIL,2698077,Guayas
...,...,...,...,...,...,...,...
75200,1,16,1604,1,ARAJUNO,7989,Pastaza
75201,1,16,1604,1,ARAJUNO,7989,Pastaza
75202,1,16,1604,1,ARAJUNO,7989,Pastaza
75216,1,16,1603,0,SANTA CLARA,4110,Pastaza


Sexo: 2


,sexo,cod_provincia,cod_canton,violenta,canton,poblacion,nom_provincia
9127,2,9,901,0,GUAYAQUIL,2698077,Guayas
9128,2,9,901,1,GUAYAQUIL,2698077,Guayas
9129,2,9,901,0,GUAYAQUIL,2698077,Guayas
9130,2,9,901,0,GUAYAQUIL,2698077,Guayas
9131,2,9,901,0,GUAYAQUIL,2698077,Guayas
...,...,...,...,...,...,...,...
75213,2,16,1604,0,ARAJUNO,7989,Pastaza
75214,2,16,1604,0,ARAJUNO,7989,Pastaza
75215,2,16,1604,0,ARAJUNO,7989,Pastaza
75218,2,16,1603,0,SANTA CLARA,4110,Pastaza


### Métodos `aggregate`, `filter`, `apply` y `transform`.

La clase `DataFrameGroupBy` tiene cuatro métodos que sirven para implementar eficientemente una gran variedad de operaciones útiles antes de combinar los datos seccionados.

El método `aggregate` permite especificar una lista de funciones de agregación que se aplicarán a cada una de las columnas seleccionadas del `DataFrameGroupBy`:

In [15]:
# aplicar las funciones de agregación min, max y sum a las columnas sexo, cod_canton, violenta y poblacion
# (las respuestas en la mayoría de los casos carecen de significado práctico)
display(dfmuertes2019ext.groupby('nom_provincia')[['sexo', 'cod_canton', 'violenta', 
                                                   'poblacion']].aggregate([min, max, sum]))

sexo            cod_canton                  \
                                min max    sum        min   max       sum   
nom_provincia                                                               
Azuay                             1   2   6135        101   115    419027   
Bolívar                           1   2   1261        201   207    172674   
Carchi                            1   2   1084        401   406    305710   
Cañar                             1   2   1696        301   307    353614   
Chimborazo                        1   2   3616        601   610   1476706   
Cotopaxi                          1   2   2803        501   507    971772   
El Oro                            1   2   4402        701   714   2206872   
Esmeraldas                        1   2   2553        801   807   1447386   
Galápagos                         1   2     58       2001  2003     94110   
Guayas                            1   2  31145        901   928  19563395   
Imbabura                          1   2   3019       1001  1006   2050282   
Loja                              1   2   3658       1101  1116   2738199   
Los Ríos                          1   2   5224       1201  1213   4472839   
Manabí                            1   2   9664       1301  1322   8881588   
Morona Santiago                   1   2    775       1401  1412    749633   
Napo                              1   2    585       1501  1509    620471   
Orellana                          1   2    617       2201  2204    979861   
Pastaza                           1   2    509       1601  1604    570067   
Pichincha                         1   2  19715       1701  1709  22720102   
Santa Elena                       1   2   1899       2401  2403   3203854   
Santo Domingo de los Tsáchilas    1   2   2898       2301  2301   4742361   
Sucumbíos                         1   2    941       2101  2107   1414707   
Tungurahua                        1   2   4153       1801  1809   5107647   
Zamora Chinchipe                  1   2    394       1901  1909    525503   

                               violenta           poblacion           \
                                    min max   sum       min      max   
nom_provincia                                                          
Azuay                                 0   1   465      3105   625775   
Bolívar                               0   1   124      7325   107590   
Carchi                                0   1   118      8840   101234   
Cañar                                 0   1   170      6387    85030   
Chimborazo                            0   1   346      6975   261360   
Cotopaxi                              0   1   369     23276   202878   
El Oro                                0   1   397      2405   286120   
Esmeraldas                            0   1   298     31025   216901   
Galápagos                             0   1     7      2995    19852   
Guayas                                0   1  2076     12944  2698077   
Imbabura                              0   1   234     13315   217856   
Loja                                  0   1   199      4239   269017   
Los Ríos                              0   1   560     20092   210461   
Manabí                                0   1   685     10222   319185   
Morona Santiago                       0   1   116      4901    56679   
Napo                                  0   1    83      4401    77502   
Orellana                              0   1   104      3765    91933   
Pastaza                               0   1    65      4110    82221   
Pichincha                             0   1  1703     16771  2735987   
Santa Elena                           0   1   127     92017   184642   
Santo Domingo de los Tsáchilas        0   1   276    450694   450694   
Sucumbíos                             0   1   152      3788   117048   
Tungurahua                            0   1   394      7315   382941   
Zamora Chinchipe                      0   1    48      5768    32172   

      

Alternativamente, la función `aggregate` puede recibir como parámetro un diccionario cuyas claves sean las columnas a seleccionar del `DataFrameGroupBy`, y cuyos valores sean las funciones de agregación a aplicar sobre cada columna:

In [16]:
# total de fallecidos por provincia y número de muertes violentas
display(dfmuertes2019ext.groupby('nom_provincia'
                                ).aggregate({'cod_provincia' : 'count', 'violenta' : 'sum'}))

,cod_provincia,violenta
nom_provincia,,
Azuay,4100,465
Bolívar,853,124
Carchi,760,118
Cañar,1169,170
Chimborazo,2451,346
Cotopaxi,1934,369
El Oro,3131,397
Esmeraldas,1804,298
Galápagos,47,7


La función `filter` recibe como parámetro el nombre de una *función de filtrado*. Esta función debe recibir un DataFrame como parámetro y retornar el valor de `True` o `False`. La función será llamada con cada uno de los DataFrames del objeto `DataFrameGroupBy` y solamente aquellos para los cuales el valor de retorno sea verdadero serán incluidos en la combinación. De esta manera, es posible implementar una condición de filtrado a nivel de grupos.

Por ejemplo, suponer que se quieren listar las provincias con más de 5.000 fallecidos en el 2019, conjuntamente con el número de muertes violentas:

In [17]:
# total de fallecidos por provincia y número de muertes violentas
# solamente para aquellas provincias con más de 5000 fallecimientos
def tiene_muchos_fallecidos(df):
    return df['cod_provincia'].count() > 5000

# primero filtramos los registros que nos interesan
df1= dfmuertes2019ext.groupby('nom_provincia').filter(tiene_muchos_fallecidos)
display(df1)
# luego agrupamos nuevamente y agregamos
display(df1.groupby('nom_provincia').aggregate({'cod_provincia' : 'count', 'violenta' : 'sum'}))

,sexo,cod_provincia,cod_canton,violenta,canton,poblacion,nom_provincia
0,1,9,901,1,GUAYAQUIL,2698077,Guayas
1,1,9,901,0,GUAYAQUIL,2698077,Guayas
2,1,9,901,0,GUAYAQUIL,2698077,Guayas
3,1,9,901,1,GUAYAQUIL,2698077,Guayas
4,1,9,901,0,GUAYAQUIL,2698077,Guayas
...,...,...,...,...,...,...,...
74976,2,13,1316,0,24 DE MAYO,28731,Manabí
74977,2,13,1316,0,24 DE MAYO,28731,Manabí
74978,2,13,1316,0,24 DE MAYO,28731,Manabí
74979,2,13,1316,0,24 DE MAYO,28731,Manabí


,cod_provincia,violenta
nom_provincia,,
Guayas,21654,2076
Manabí,6797,685
Pichincha,13355,1703


La función `groupby` puede aplicarse a varias columnas, en cuyo caso el DataFrame resultante estará indexado por un multi-índice:

In [18]:
dfxcanton = dfmuertes2019ext.groupby(['canton', 'sexo', 'violenta']).aggregate(
                                        {'cod_canton' : 'count', 'poblacion' : 'max', 'nom_provincia' : 'max'})

# renombrar columna "cod_canton" como "muertes"
dfxcanton.rename(columns={'cod_canton' : 'muertes'}, inplace=True)
display(dfxcanton)

muertes  poblacion nom_provincia
canton     sexo violenta                                  
24 DE MAYO 1    0              61      28731        Manabí
                1              15      28731        Manabí
           2    0              43      28731        Manabí
                1               1      28731        Manabí
AGUARICO   1    0              16       3765      Orellana
...                           ...        ...           ...
ZAPOTILLO  2    0              21      14194          Loja
                1               4      14194          Loja
ZARUMA     1    0              51      25651        El Oro
                1              15      25651        El Oro
           2    0              35      25651        El Oro

[827 rows x 3 columns]

Para los ejemplos que siguen, nos conviene eliminar el multi-índice del DataFrame y pasar la información correspondiente a columnas:

In [19]:
dfxcanton.reset_index(inplace=True)
display(dfxcanton)

,canton,sexo,violenta,muertes,poblacion,nom_provincia
0,24 DE MAYO,1,0,61,28731,Manabí
1,24 DE MAYO,1,1,15,28731,Manabí
2,24 DE MAYO,2,0,43,28731,Manabí
3,24 DE MAYO,2,1,1,28731,Manabí
4,AGUARICO,1,0,16,3765,Orellana
...,...,...,...,...,...,...
822,ZAPOTILLO,2,0,21,14194,Loja
823,ZAPOTILLO,2,1,4,14194,Loja
824,ZARUMA,1,0,51,25651,El Oro
825,ZARUMA,1,1,15,25651,El Oro


En `dfxcanton` tenemos registros del número de muertes violentas y no violentas por cada sexo y por cada cantón. Suponer que para cada cantón queremos establecer el *porcentaje* de muertes violentas y no violentas por cada sexo. Para ello, necesitamos dividir los valores de la columna muertes por los totales de muertos por cantón.

La función `apply` recibe como parámetro una función de transformación. Esta función se aplicará a cada uno de los DataFrames de grupos contenidos en el `DataFrameGroupBy`. La función de transformación debe retornar un DataFrame, una serie o un escalar. Los objetos retornados para todos los grupos serán combinados para obtener la respuesta final.

In [20]:
def calcular_porcentaje(df):
    # df representa a un DataFrame de grupo
    # agregamos una nueva columna con el valor de muertes dividido para el total del grupo
    df['porcentaje'] = df['muertes'] / df['muertes'].sum() 
    return df
 
display(dfxcanton.groupby('canton').apply(calcular_porcentaje) )

,canton,sexo,violenta,muertes,poblacion,nom_provincia,porcentaje
0,24 DE MAYO,1,0,61,28731,Manabí,0.508333
1,24 DE MAYO,1,1,15,28731,Manabí,0.125000
2,24 DE MAYO,2,0,43,28731,Manabí,0.358333
3,24 DE MAYO,2,1,1,28731,Manabí,0.008333
4,AGUARICO,1,0,16,3765,Orellana,0.615385
...,...,...,...,...,...,...,...
822,ZAPOTILLO,2,0,21,14194,Loja,0.381818
823,ZAPOTILLO,2,1,4,14194,Loja,0.072727
824,ZARUMA,1,0,51,25651,El Oro,0.504950
825,ZARUMA,1,1,15,25651,El Oro,0.148515


Si la función de transformación retorna una serie, las series para los distintos grupos se combinan como filas en un DataFrame:

In [21]:
def por_tipo_y_sexo(df):
    return pd.Series({'Provincia' : df['nom_provincia'].max(),
                      'Hombres' : df[df['sexo']==1]['muertes'].sum(),
                      'Mujeres' : df[df['sexo']==2]['muertes'].sum(),
                      'Violentas' : df[df['violenta']==1]['muertes'].sum(),
                      'No violentas' : df[df['violenta']==0]['muertes'].sum(),
                      'Total' : df['muertes'].sum(),
                      'Poblacion' : df['poblacion'].min(),
                      'Por 1000h' : df['muertes'].sum() / df['poblacion'].min() * 1000})

dfxcanton2 = dfxcanton.groupby('canton').apply(por_tipo_y_sexo)
display(dfxcanton2)

,Provincia,Hombres,Mujeres,Violentas,No violentas,Total,Poblacion,Por 1000h
canton,,,,,,,,
24 DE MAYO,Manabí,76,44,16,104,120,28731,4.176673
AGUARICO,Orellana,17,9,2,24,26,3765,6.905710
ALAUSI,Chimborazo,110,90,32,168,200,45229,4.421942
ALFREDO BAQUERIZO MORENO,Guayas,64,27,19,72,91,31491,2.889715
AMBATO,Tungurahua,1139,1001,274,1866,2140,382941,5.588328
...,...,...,...,...,...,...,...,...
YAGUACHI,Guayas,104,60,36,128,164,76648,2.139651
YANTZAZA,Zamora Chinchipe,39,28,10,57,67,25708,2.606193
ZAMORA,Zamora Chinchipe,45,33,16,62,78,32172,2.424468


Por último, si la función de transformación retorna un valor escalar, los valores para los distintos grupos se combinan para formar una serie:

In [22]:
def total_muertes(df):
    return df['muertes'].sum()

display(dfxcanton.groupby('canton').apply(total_muertes))

canton
24 DE MAYO                   120
AGUARICO                      26
ALAUSI                       200
ALFREDO BAQUERIZO MORENO      91
AMBATO                      2140
                            ... 
YAGUACHI                     164
YANTZAZA                      67
ZAMORA                        78
ZAPOTILLO                     55
ZARUMA                       101
Length: 217, dtype: int64

La función de transformación utilizada con `apply` es muy general y no necesariamente debe calcular algún tipo de agregación. Por ejemplo, puede usarse esta función para construir un DataFrame que contenga la información de los tres cantones con más muertes violentas en cada provincia:

In [23]:
# ordenar cada DataFrame por la columna Violentas y retornar tres primeras filas
def mas_muertes_violentas(df):
    df2 = df.sort_values(by='Violentas', ascending=False)
    return df2.head(3)

display(dfxcanton2.reset_index().groupby('Provincia').apply(mas_muertes_violentas))

canton         Provincia  Hombres  \
Provincia                                                                
Azuay            44                  CUENCA             Azuay     1584   
                 69                GUALACEO             Azuay      116   
                 23   CAMILO PONCE ENRIQUEZ             Azuay       41   
Bolívar          73                GUARANDA           Bolívar      250   
                 50               ECHEANDIA           Bolívar       37   
...                                     ...               ...      ...   
Tungurahua       176   SAN PEDRO DE PELILEO        Tungurahua      112   
                 186    SANTIAGO DE PILLARO        Tungurahua       90   
Zamora Chinchipe 214                 ZAMORA  Zamora Chinchipe       45   
                 213               YANTZAZA  Zamora Chinchipe       39   
                 36               CHINCHIPE  Zamora Chinchipe       19   

                      Mujeres  Violentas  No violentas  Total  Poblacion  \
Provincia                                                                  
Azuay            44      1602        313          2873   3186     625775   
                 69       122         31           207    238      48702   
                 23         8         26            23     49      34774   
Bolívar          73       231         57           424    481     107590   
                 50        20         17            40     57      13956   
...                       ...        ...           ...    ...        ...   
Tungurahua       176      107         31           188    219      66039   
                 186       85         24           151    175      43051   
Zamora Chinchipe 214       33         16            62     78      32172   
                 213       28         10            57     67      25708   
                 36        11          6            24     30      10588   

                      Por 1000h  
Provincia                        
Azuay            44    5.091287  
                 69    4.886863  
                 23    1.409099  
Bolívar          73    4.470676  
                 50    4.084265  
...                         ...  
Tungurahua       176   3.316222  
                 186   4.064946  
Zamora Chinchipe 214   2.424468  
                 213   2.606193  
                 36    2.833396  

[70 rows x 9 columns]

Finalmente, el método `transform` se usa de manera parecida al método `apply`, aunque es más limitado. Al igual que en el caso de `apply`, el método `transform` recibe como parámetro una función de transformación que se aplicará a cada uno de los DataFrames de los grupos. Sin embargo, en este caso la función no puede alterar el tamaño del DataFrame.

Un uso común del método `transform` es para centrar observaciones respecto a medias grupales. Por ejemplo, suponer que agregamos al DataFrame `dfxcanton2` cuatro columnas adicionales que nos indican el número de muertes por cada 1000 habitantes, segregadas por sexo, y entre violentas y no violentas:

In [24]:
dfxcanton2['Hombres / 1000 h']= dfxcanton2['Hombres'] / dfxcanton2['Poblacion'] * 1000
dfxcanton2['Mujeres / 1000 h']= dfxcanton2['Mujeres'] / dfxcanton2['Poblacion'] * 1000
dfxcanton2['Violentas / 1000 h']= dfxcanton2['Violentas'] / dfxcanton2['Poblacion'] * 1000
dfxcanton2['No Violentas / 1000 h']= dfxcanton2['No violentas'] / dfxcanton2['Poblacion'] * 1000
display(dfxcanton2)


,Provincia,Hombres,Mujeres,Violentas,No violentas,Total,Poblacion,Por 1000h,Hombres / 1000 h,Mujeres / 1000 h,Violentas / 1000 h,No Violentas / 1000 h
canton,,,,,,,,,,,,
24 DE MAYO,Manabí,76,44,16,104,120,28731,4.176673,2.645226,1.531447,0.556890,3.619784
AGUARICO,Orellana,17,9,2,24,26,3765,6.905710,4.515272,2.390438,0.531208,6.374502
ALAUSI,Chimborazo,110,90,32,168,200,45229,4.421942,2.432068,1.989874,0.707511,3.714431
ALFREDO BAQUERIZO MORENO,Guayas,64,27,19,72,91,31491,2.889715,2.032327,0.857388,0.603347,2.286368
AMBATO,Tungurahua,1139,1001,274,1866,2140,382941,5.588328,2.974349,2.613980,0.715515,4.872813
...,...,...,...,...,...,...,...,...,...,...,...,...
YAGUACHI,Guayas,104,60,36,128,164,76648,2.139651,1.356852,0.782799,0.469680,1.669972
YANTZAZA,Zamora Chinchipe,39,28,10,57,67,25708,2.606193,1.517037,1.089155,0.388984,2.217209
ZAMORA,Zamora Chinchipe,45,33,16,62,78,32172,2.424468,1.398732,1.025737,0.497327,1.927142


Suponer ahora que queremos centrar los valores de muertes por 1000 habitantes en cada cantón respecto a los valores de las medias provinciales correspondientes. Para hacer esto definimos una función `centrar_provincia` y llamamos al método `transform` para aplicarla sobre datos cantonales agrupados por provincia:

In [25]:
# centrar números de muertes por 1000 h respecto a los promedios provinciales
def centrar_provincia(df):
    return df - df.mean()

display(dfxcanton2.groupby('Provincia')[['Por 1000h', 'Hombres / 1000 h', 'Mujeres / 1000 h', 
                                         'Violentas / 1000 h', 'No Violentas / 1000 h']].transform(centrar_provincia))

,Por 1000h,Hombres / 1000 h,Mujeres / 1000 h,Violentas / 1000 h,No Violentas / 1000 h
canton,,,,,
24 DE MAYO,0.002417,0.212309,-0.209892,0.012923,-0.010505
AGUARICO,3.177339,2.140124,1.037215,-0.106818,3.284156
ALAUSI,0.396971,0.336928,0.060043,-0.027163,0.424134
ALFREDO BAQUERIZO MORENO,-0.303673,0.124066,-0.427739,0.131220,-0.434893
AMBATO,1.998257,0.971601,1.026656,0.029364,1.968893
...,...,...,...,...,...
YAGUACHI,-1.053736,-0.551409,-0.502327,-0.002447,-1.051289
YANTZAZA,0.343589,0.230707,0.112882,0.010380,0.333208
ZAMORA,0.161864,0.112401,0.049463,0.118723,0.043141


Notar que la función `transform` no altera el número de filas del DataFrame.

Por otra parte, la tarea anterior puede realizarse también utilizando el método `apply`:

In [26]:
display(dfxcanton2.groupby('Provincia')[['Por 1000h', 'Hombres / 1000 h', 'Mujeres / 1000 h', 
                                         'Violentas / 1000 h', 'No Violentas / 1000 h']].apply(centrar_provincia))

,Por 1000h,Hombres / 1000 h,Mujeres / 1000 h,Violentas / 1000 h,No Violentas / 1000 h
canton,,,,,
24 DE MAYO,0.002417,0.212309,-0.209892,0.012923,-0.010505
AGUARICO,3.177339,2.140124,1.037215,-0.106818,3.284156
ALAUSI,0.396971,0.336928,0.060043,-0.027163,0.424134
ALFREDO BAQUERIZO MORENO,-0.303673,0.124066,-0.427739,0.131220,-0.434893
AMBATO,1.998257,0.971601,1.026656,0.029364,1.968893
...,...,...,...,...,...
YAGUACHI,-1.053736,-0.551409,-0.502327,-0.002447,-1.051289
YANTZAZA,0.343589,0.230707,0.112882,0.010380,0.333208
ZAMORA,0.161864,0.112401,0.049463,0.118723,0.043141


**Funciones lambda.** Cuando las funciones que se pasan a los métodos `filter`, `apply` o `transform` son simples, en lugar de definirlas por separado suele usar la sintaxis en-línea de las funciones lambda:

In [27]:
display(dfxcanton2.groupby('Provincia')[['Por 1000h', 'Hombres / 1000 h', 
                                         'Mujeres / 1000 h', 'Violentas / 1000 h', 
                                         'No Violentas / 1000 h']].transform(lambda df : df - df.mean()))

,Por 1000h,Hombres / 1000 h,Mujeres / 1000 h,Violentas / 1000 h,No Violentas / 1000 h
canton,,,,,
24 DE MAYO,0.002417,0.212309,-0.209892,0.012923,-0.010505
AGUARICO,3.177339,2.140124,1.037215,-0.106818,3.284156
ALAUSI,0.396971,0.336928,0.060043,-0.027163,0.424134
ALFREDO BAQUERIZO MORENO,-0.303673,0.124066,-0.427739,0.131220,-0.434893
AMBATO,1.998257,0.971601,1.026656,0.029364,1.968893
...,...,...,...,...,...
YAGUACHI,-1.053736,-0.551409,-0.502327,-0.002447,-1.051289
YANTZAZA,0.343589,0.230707,0.112882,0.010380,0.333208
ZAMORA,0.161864,0.112401,0.049463,0.118723,0.043141


### Especificando la clave de agrupamiento

Por defecto, `groupby` realiza la operación de seccionar (*split*) en base a los valores de una columna del DataFrame cuyo nombre se especifica como parámetro. Es posible agrupar también por los valores del índice:

In [28]:
dfxcanton.set_index('canton', inplace=True)
display(dfxcanton)
display(dfxcanton.groupby(dfxcanton.index)[['muertes']].sum())

,sexo,violenta,muertes,poblacion,nom_provincia
canton,,,,,
24 DE MAYO,1,0,61,28731,Manabí
24 DE MAYO,1,1,15,28731,Manabí
24 DE MAYO,2,0,43,28731,Manabí
24 DE MAYO,2,1,1,28731,Manabí
AGUARICO,1,0,16,3765,Orellana
...,...,...,...,...,...
ZAPOTILLO,2,0,21,14194,Loja
ZAPOTILLO,2,1,4,14194,Loja
ZARUMA,1,0,51,25651,El Oro


,muertes
canton,
24 DE MAYO,120
AGUARICO,26
ALAUSI,200
ALFREDO BAQUERIZO MORENO,91
AMBATO,2140
...,...
YAGUACHI,164
YANTZAZA,67
ZAMORA,78


También es posible especificar una lista o serie con los valores a usar en el agrupamiento, la misma que debe tener un tamaño igual al número de filas del DataFrame. Por ejemplo, generemos una serie duplicando los valores de la columna `sexo` y sumándoles los valores de la columna `violencia`. Los elementos de esta serie pueden tomar cuatro valores posibles (2, 3, 4 o 5), correspondientes a las cuatro combinaciones de las dos columnas:

In [29]:
s=2*dfxcanton['sexo'] + dfxcanton['violenta']
print(s)

canton
24 DE MAYO    2
24 DE MAYO    3
24 DE MAYO    4
24 DE MAYO    5
AGUARICO      2
             ..
ZAPOTILLO     4
ZAPOTILLO     5
ZARUMA        2
ZARUMA        3
ZARUMA        4
Length: 827, dtype: int64


Agrupemos ahora `dfxcanton` por los valores de esta serie y calculemos el número total de muertes en cada categoría:

In [30]:
display(dfxcanton.groupby(s)['muertes'].sum())

2    34482
3     7154
4    31622
5     1962
Name: muertes, dtype: int64

Para mayor información sobre las funciones de agrupamiento y agregación, se puede consultar la documentación del sitio web de `pandas`: <https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html>.